# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

download_week3_resources()

File data/train.tsv is already downloaded.
File data/validation.tsv is already downloaded.
File data/test.tsv is already downloaded.
File data/test_embeddings.tsv is already downloaded.


### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [2]:
from grader import Grader

In [3]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. You need to download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [4]:
import gensim

In [5]:
from gensim.models import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True, limit=500000)
######### YOUR CODE HERE #############

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [6]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [7]:
print(check_embeddings(wv_embeddings))

## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [8]:
import numpy as np

In [9]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    output = np.zeros(dim)
    strings = question.split()
    emb_count = 0.0
    
    for string in strings:
        if string in embeddings:
            emb = embeddings[string]
            output += emb
            emb_count += 1
    
    if emb_count > 0:
        return output/emb_count
    else:
        return output    

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [10]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [11]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [12]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891059
-0.0287272135417
0.0460561116536
0.0852593315972
0.0243055555556
-0.0729031032986
0.0...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [14]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    out = [x<=k for x in dup_ranks]
    
    return sum(out)/len(out)

Test your code on the tiny examples:

In [15]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [16]:
print(test_hits())

Basic test are passed.


In [17]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    total_val = 0
    for rank in dup_ranks:
        out = rank <= k
        out = np.log(2) / np.log(1+rank) * out
        total_val += out
    
    return total_val/len(dup_ranks)

In [18]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [19]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [20]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [21]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....


In [22]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.815464876786
0.5
0.815464876786
0.333333333333
0.54364325119
0.710309917857
0.1
0.16309297...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [23]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [40]:
validation = read_corpus('data/validation.tsv')  ######### YOUR CODE HERE #############

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [25]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    ref_vec = question_to_vec(question, embeddings, dim).reshape([1,-1])
    cand_vec = np.array([question_to_vec(q, embeddings, dim) for q in candidates])
    cand_vec = cand_vec.reshape([len(candidates),-1])
    ind = np.argsort(cosine_similarity(ref_vec, cand_vec))
    
    return [(x, candidates[x]) for x in ind[0]][::-1]
    
    """
    # What did not work!
    q_vec = np.asarray([question_to_vec(question, embeddings, dim=300)])
    cand_vec = np.asarray([question_to_vec(candidate, embeddings, dim=300) for candidate in candidates])
    similarity_index = cosine_similarity(q_vec, cand_vec)[0]
    indices = np.argsort(-1.0*similarity_index)
    unordered_list = [(u,v) for u, v in enumerate(candidates)]
    ordered_list = [x for _,x in sorted(zip(indices,unordered_list))]
    return ordered_list
    """

Test your code on the tiny examples:

In [26]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [27]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [ ]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [129]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [28]:
from util import text_prepare

Now transform all the questions from the validation set:

In [41]:
prepared_validation = []
for line in validation:
    prepared_validation.append([text_prepare(q) for q in line])

Let's evaluate the approach again after the preparation:

In [ ]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

Now, prepare also train and test data, because you will need it in the future:

In [29]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [30]:
######################################
######### YOUR CODE HERE #############
######################################

prepare_file('data/train.tsv', 'data/train_prepped.tsv')
prepare_file('data/test.tsv', 'data/test_prepped.tsv')

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [31]:
from util import matrix_to_string

In [32]:
w2v_ranks_results = []
prepared_test_data = 'data/test_prepped.tsv'   ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour.

In [33]:
######### TRAINING HAPPENING HERE #############

!starspace train -trainFile "data/train_prepped.tsv" \
    -model starspace_embedding \
    -trainMode 3 \
    -adagrad true \
    -ngrams 1 \
    -epoch 5 \
    -dim 100 \
    -similarity "cosine" \
    -minCount 2 \
    -verbose true \
    -fileFormat labelDoc \
    -negSearchLimit 10 \
    -lr 0.05 \
    -thread 4

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 4
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/train_prepped.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : data/train_prepped.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01


Epoch: 31.9%  lr: 0.046677  loss: 0.013675  eta: 0h44m  tot: 0h3m1s  (6.4%))1%  lr: 0.050000  loss: 0.096849  eta: 0h53m  tot: 0h0m0s  (0.0%)0.2%  lr: 0.049990  loss: 0.083848  eta: 0h51m  tot: 0h0m1s  (0.0%)0.4%  lr: 0.049970  loss: 0.066353  eta: 0h50m  tot: 0h0m2s  (0.1%)0.5%  lr: 0.049970  loss: 0.063756  eta: 0h50m  tot: 0h0m3s  (0.1%)0.5%  lr: 0.049970  loss: 0.062025  eta: 0h50m  tot: 0h0m3s  (0.1%)0.6%  lr: 0.049940  loss: 0.058346  eta: 0h49m  tot: 0h0m3s  (0.1%)0.7%  lr: 0.049930  loss: 0.057056  eta: 0h49m  tot: 0h0m3s  (0.1%)%  lr: 0.049860  loss: 0.047368  eta: 0h49m  tot: 0h0m7s  (0.2%)%  lr: 0.049850  loss: 0.046807  eta: 0h49m  tot: 0h0m7s  (0.2%)1.3%  lr: 0.049820  loss: 0.045733  eta: 0h49m  tot: 0h0m7s  (0.3%)1.3%  lr: 0.049820  loss: 0.045130  eta: 0h48m  tot: 0h0m7s  (0.3%)1.3%  lr: 0.049810  loss: 0.044686  eta: 0h48m  tot: 0h0m7s  (0.3%)1.5%  lr: 0.049790  loss: 0.043370  eta: 0h49m  tot: 0h0m8s  (0.3%)2.3%  lr: 0.049680  loss: 0.037703  eta: 0h48m  tot: 0h0m13s 

Epoch: 100.0%  lr: 0.040000  loss: 0.008810  eta: 0h38m  tot: 0h9m37s  (20.0%)5.6%  lr: 0.043043  loss: 0.010267  eta: 0h41m  tot: 0h6m20s  (13.1%)65.9%  lr: 0.043023  loss: 0.010247  eta: 0h41m  tot: 0h6m22s  (13.2%)66.0%  lr: 0.043013  loss: 0.010242  eta: 0h41m  tot: 0h6m22s  (13.2%)66.1%  lr: 0.043013  loss: 0.010232  eta: 0h41m  tot: 0h6m23s  (13.2%)%  lr: 0.042983  loss: 0.010220  eta: 0h41m  tot: 0h6m25s  (13.3%)66.5%  lr: 0.042983  loss: 0.010214  eta: 0h41m  tot: 0h6m25s  (13.3%)66.7%  lr: 0.042973  loss: 0.010203  eta: 0h41m  tot: 0h6m27s  (13.3%)66.8%  lr: 0.042953  loss: 0.010200  eta: 0h41m  tot: 0h6m27s  (13.4%)66.9%  lr: 0.042943  loss: 0.010198  eta: 0h41m  tot: 0h6m27s  (13.4%)67.4%  lr: 0.042893  loss: 0.010168  eta: 0h41m  tot: 0h6m31s  (13.5%)67.5%  lr: 0.042853  loss: 0.010159  eta: 0h41m  tot: 0h6m32s  (13.5%)67.9%  lr: 0.042833  loss: 0.010138  eta: 0h41m  tot: 0h6m34s  (13.6%)68.1%  lr: 0.042823  loss: 0.010131  eta: 0h41m  tot: 0h6m35s  (13.6%)68.4%  lr: 0.0427

Epoch: 29.4%  lr: 0.036857  loss: 0.002649  eta: 0h37m  tot: 0h12m33s  (25.9%)9%  lr: 0.039860  loss: 0.002882  eta: 0h40m  tot: 0h9m42s  (20.2%)1.3%  lr: 0.039820  loss: 0.002736  eta: 0h40m  tot: 0h9m45s  (20.3%)1.5%  lr: 0.039810  loss: 0.002672  eta: 0h40m  tot: 0h9m46s  (20.3%)1.5%  lr: 0.039810  loss: 0.002645  eta: 0h40m  tot: 0h9m46s  (20.3%)2.0%  lr: 0.039760  loss: 0.002652  eta: 0h40m  tot: 0h9m49s  (20.4%)2.1%  lr: 0.039760  loss: 0.002728  eta: 0h40m  tot: 0h9m50s  (20.4%)2.4%  lr: 0.039720  loss: 0.002732  eta: 0h39m  tot: 0h9m51s  (20.5%)2.7%  lr: 0.039680  loss: 0.002711  eta: 0h39m  tot: 0h9m53s  (20.5%)3.0%  lr: 0.039640  loss: 0.002640  eta: 0h39m  tot: 0h9m55s  (20.6%)3.6%  lr: 0.039590  loss: 0.002706  eta: 0h39m  tot: 0h9m58s  (20.7%)3.8%  lr: 0.039570  loss: 0.002729  eta: 0h39m  tot: 0h9m59s  (20.8%)4.1%  lr: 0.039550  loss: 0.002686  eta: 0h39m  tot: 0h10m1s  (20.8%)4.3%  lr: 0.039510  loss: 0.002691  eta: 0h39m  tot: 0h10m2s  (20.9%)4.5%  lr: 0.039500  loss: 0

Epoch: 87.7%  lr: 0.031101  loss: 0.002696  eta: 0h31m  tot: 0h18m22s  (37.5%)59.5%  lr: 0.033634  loss: 0.002702  eta: 0h33m  tot: 0h15m32s  (31.9%)59.7%  lr: 0.033634  loss: 0.002702  eta: 0h33m  tot: 0h15m33s  (31.9%)59.8%  lr: 0.033614  loss: 0.002703  eta: 0h33m  tot: 0h15m34s  (32.0%)60.0%  lr: 0.033604  loss: 0.002703  eta: 0h33m  tot: 0h15m36s  (32.0%)60.3%  lr: 0.033604  loss: 0.002702  eta: 0h33m  tot: 0h15m37s  (32.1%)60.3%  lr: 0.033584  loss: 0.002702  eta: 0h33m  tot: 0h15m37s  (32.1%)61.0%  lr: 0.033544  loss: 0.002703  eta: 0h33m  tot: 0h15m41s  (32.2%)61.8%  lr: 0.033434  loss: 0.002696  eta: 0h33m  tot: 0h15m46s  (32.4%)61.9%  lr: 0.033434  loss: 0.002699  eta: 0h33m  tot: 0h15m47s  (32.4%)62.0%  lr: 0.033434  loss: 0.002700  eta: 0h33m  tot: 0h15m47s  (32.4%)62.1%  lr: 0.033424  loss: 0.002701  eta: 0h33m  tot: 0h15m48s  (32.4%)62.4%  lr: 0.033424  loss: 0.002701  eta: 0h33m  tot: 0h15m50s  (32.5%)62.4%  lr: 0.033424  loss: 0.002702  eta: 0h33m  tot: 0h15m50s  (32.5%

...

Epoch: 90.1%  lr: 0.010711  loss: 0.001574  eta: 0h10m  tot: 0h37m59s  (78.0%)59.3%  lr: 0.013654  loss: 0.001556  eta: 0h14m  tot: 0h34m56s  (71.9%)59.6%  lr: 0.013654  loss: 0.001556  eta: 0h14m  tot: 0h34m57s  (71.9%)59.8%  lr: 0.013654  loss: 0.001555  eta: 0h13m  tot: 0h34m58s  (72.0%)60.1%  lr: 0.013624  loss: 0.001555  eta: 0h13m  tot: 0h35m0s  (72.0%)60.1%  lr: 0.013614  loss: 0.001555  eta: 0h13m  tot: 0h35m1s  (72.0%)60.3%  lr: 0.013604  loss: 0.001554  eta: 0h13m  tot: 0h35m2s  (72.1%)60.4%  lr: 0.013604  loss: 0.001556  eta: 0h13m  tot: 0h35m2s  (72.1%)60.7%  lr: 0.013574  loss: 0.001555  eta: 0h13m  tot: 0h35m4s  (72.1%)61.0%  lr: 0.013524  loss: 0.001555  eta: 0h13m  tot: 0h35m6s  (72.2%)61.1%  lr: 0.013524  loss: 0.001555  eta: 0h13m  tot: 0h35m6s  (72.2%)61.4%  lr: 0.013504  loss: 0.001555  eta: 0h13m  tot: 0h35m8s  (72.3%)61.5%  lr: 0.013504  loss: 0.001555  eta: 0h13m  tot: 0h35m9s  (72.3%)62.0%  lr: 0.013464  loss: 0.001553  eta: 0h13m  tot: 0h35m12s  (72.4%)62.5%  l

Epoch: 100.0%  lr: 0.010000  loss: 0.001574  eta: 0h9m  tot: 0h38m49s  (80.0%)0.7%  lr: 0.010681  loss: 0.001574  eta: 0h10m  tot: 0h38m3s  (78.1%)90.8%  lr: 0.010661  loss: 0.001574  eta: 0h10m  tot: 0h38m4s  (78.2%)91.0%  lr: 0.010641  loss: 0.001574  eta: 0h10m  tot: 0h38m5s  (78.2%)91.0%  lr: 0.010641  loss: 0.001574  eta: 0h10m  tot: 0h38m5s  (78.2%)91.2%  lr: 0.010621  loss: 0.001574  eta: 0h10m  tot: 0h38m6s  (78.2%)91.8%  lr: 0.010591  loss: 0.001576  eta: 0h10m  tot: 0h38m10s  (78.4%)92.0%  lr: 0.010561  loss: 0.001576  eta: 0h10m  tot: 0h38m11s  (78.4%)92.1%  lr: 0.010561  loss: 0.001576  eta: 0h10m  tot: 0h38m11s  (78.4%)92.7%  lr: 0.010511  loss: 0.001574  eta: 0h10m  tot: 0h38m15s  (78.5%)92.8%  lr: 0.010511  loss: 0.001574  eta: 0h10m  tot: 0h38m15s  (78.6%)92.9%  lr: 0.010481  loss: 0.001573  eta: 0h10m  tot: 0h38m16s  (78.6%)93.0%  lr: 0.010481  loss: 0.001573  eta: 0h10m  tot: 0h38m16s  (78.6%)93.4%  lr: 0.010461  loss: 0.001574  eta: 0h10m  tot: 0h38m19s  (78.7%)94.0%

Epoch: 31.1%  lr: 0.006647  loss: 0.001387  eta: 0h6m  tot: 0h41m52s  (86.2%).3%  lr: 0.009890  loss: 0.001346  eta: 0h9m  tot: 0h38m57s  (80.3%)1.8%  lr: 0.009770  loss: 0.001373  eta: 0h9m  tot: 0h39m0s  (80.4%)80.4%)2.2%  lr: 0.009740  loss: 0.001362  eta: 0h9m  tot: 0h39m1s  (80.4%)3.7%  lr: 0.009580  loss: 0.001331  eta: 0h9m  tot: 0h39m10s  (80.7%)3.9%  lr: 0.009580  loss: 0.001307  eta: 0h9m  tot: 0h39m11s  (80.8%)3.9%  lr: 0.009580  loss: 0.001317  eta: 0h9m  tot: 0h39m12s  (80.8%)4.1%  lr: 0.009570  loss: 0.001306  eta: 0h9m  tot: 0h39m13s  (80.8%)4.1%  lr: 0.009570  loss: 0.001306  eta: 0h9m  tot: 0h39m13s  (80.8%)4.7%  lr: 0.009520  loss: 0.001291  eta: 0h9m  tot: 0h39m16s  (80.9%)4.8%  lr: 0.009500  loss: 0.001302  eta: 0h9m  tot: 0h39m17s  (81.0%)5.1%  lr: 0.009459  loss: 0.001302  eta: 0h9m  tot: 0h39m19s  (81.0%)5.1%  lr: 0.009459  loss: 0.001327  eta: 0h9m  tot: 0h39m19s  (81.0%)5.3%  lr: 0.009449  loss: 0.001325  eta: 0h9m  tot: 0h39m20s  (81.1%)5.9%  lr: 0.009349  los

Epoch: 83.9%  lr: 0.001582  loss: 0.001383  eta: 0h1m  tot: 0h47m6s  (96.8%))58.4%  lr: 0.004134  loss: 0.001388  eta: 0h4m  tot: 0h44m36s  (91.7%)58.4%  lr: 0.004124  loss: 0.001389  eta: 0h4m  tot: 0h44m37s  (91.7%)58.6%  lr: 0.004124  loss: 0.001389  eta: 0h4m  tot: 0h44m37s  (91.7%)58.9%  lr: 0.004124  loss: 0.001390  eta: 0h4m  tot: 0h44m40s  (91.8%)59.4%  lr: 0.004104  loss: 0.001387  eta: 0h4m  tot: 0h44m42s  (91.9%)59.5%  lr: 0.004094  loss: 0.001386  eta: 0h4m  tot: 0h44m43s  (91.9%)59.9%  lr: 0.004054  loss: 0.001387  eta: 0h3m  tot: 0h44m45s  (92.0%)60.4%  lr: 0.004004  loss: 0.001388  eta: 0h3m  tot: 0h44m48s  (92.1%)60.6%  lr: 0.004004  loss: 0.001387  eta: 0h3m  tot: 0h44m49s  (92.1%)60.8%  lr: 0.003964  loss: 0.001387  eta: 0h3m  tot: 0h44m51s  (92.2%)60.9%  lr: 0.003964  loss: 0.001388  eta: 0h3m  tot: 0h44m51s  (92.2%)61.3%  lr: 0.003934  loss: 0.001386  eta: 0h3m  tot: 0h44m54s  (92.3%)61.6%  lr: 0.003904  loss: 0.001387  eta: 0h3m  tot: 0h44m56s  (92.3%)62.2%  lr: 0.

Epoch: 100.0%  lr: 0.000000  loss: 0.001373  eta: <1min   tot: 0h48m33s  (100.0%)  lr: 0.001582  loss: 0.001382  eta: 0h1m  tot: 0h47m7s  (96.8%)84.2%  lr: 0.001522  loss: 0.001382  eta: 0h1m  tot: 0h47m8s  (96.8%)84.4%  lr: 0.001492  loss: 0.001381  eta: 0h1m  tot: 0h47m10s  (96.9%)84.7%  lr: 0.001452  loss: 0.001381  eta: 0h1m  tot: 0h47m12s  (96.9%)85.5%  lr: 0.001381  loss: 0.001381  eta: 0h1m  tot: 0h47m16s  (97.1%)85.7%  lr: 0.001371  loss: 0.001380  eta: 0h1m  tot: 0h47m17s  (97.1%)85.9%  lr: 0.001361  loss: 0.001380  eta: 0h1m  tot: 0h47m18s  (97.2%)86.3%  lr: 0.001331  loss: 0.001379  eta: 0h1m  tot: 0h47m21s  (97.3%)86.4%  lr: 0.001331  loss: 0.001378  eta: 0h1m  tot: 0h47m21s  (97.3%)86.5%  lr: 0.001321  loss: 0.001378  eta: 0h1m  tot: 0h47m22s  (97.3%)86.5%  lr: 0.001291  loss: 0.001378  eta: 0h1m  tot: 0h47m22s  (97.3%)86.6%  lr: 0.001281  loss: 0.001377  eta: 0h1m  tot: 0h47m23s  (97.3%)87.2%  lr: 0.001151  loss: 0.001379  eta: 0h1m  tot: 0h47m26s  (97.4%)%  lr: 0.001141 

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [37]:
######### YOUR CODE HERE #############
starspace_embeddings = {}
for line in open('starspace_embedding.tsv'):
    line_elements = line.strip().split('\t')
    word = line_elements[0]
    starspace_embeddings[word] = [float(x) for x in line_elements[1:]]

In [ ]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [43]:
starspace_ranks_results = []
prepared_test_data = 'data/test_prepped.tsv' ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 88	59	75	35	17	100	98	86	94	33	25	51	20	7	99	91	8	31	47	97	80	23	4	38	87	39	76	81	90	2	40	55	48	85	7...


### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [44]:
STUDENT_EMAIL = 'prateek1592@gmail.com'
STUDENT_TOKEN = 'HJ5SPADE8oELfDzU'
grader.status()

You want to submit these parts:
Task Question2Vec: 0.019293891059
-0.0287272135417
0.0460561116536
0.0852593315972
0.0243055555556
-0.0729031032986
0.0...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....
Task DCGScore: 1.0
0.5
0.815464876786
0.5
0.815464876786
0.333333333333
0.54364325119
0.710309917857
0.1
0.16309297...
Task W2VTokenizedRanks: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...
Task StarSpaceRanks: 88	59	75	35	17	100	98	86	94	33	25	51	20	7	99	91	8	31	47	97	80	23	4	38	87	39	76	81	90	2	40	55	48	85	7...


If you want to submit these answers, run cell below

In [45]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
